# Collaborative Filtering with KNN
Author: Shiyi Wang

In [6]:
import pandas as pd
import numpy as np
from collections import Counter
from pprint import pprint

Load dataset

In [7]:
data = pd.read_pickle('../data/processed_data.pkl')
data

,date,rating,review,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,user_id,recipe_id
0,2003-02-17,4,Great with a salad. Cooked on top of stove for...,white bean green chile pepper soup,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9,3787,16642
1,2011-12-21,5,"So simple, so delicious! Great for chilly fall...",white bean green chile pepper soup,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9,95286,16642
3,2010-02-27,5,I made the Mexican topping and took it to bunk...,baked potato toppings,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13,14502,34897
4,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin...",baked potato toppings,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13,6559,34897
5,2005-05-21,4,very very sweet. after i waited the 2 days i b...,sugared raspberries,10,37449,2005-05-02,"['15-minutes-or-less', 'time-to-make', 'course...","[838.0, 1.0, 820.0, 0.0, 2.0, 0.0, 71.0]",6,"['carefully pick over the berries , removing l...",here's an old method for preserving fruit with...,"['raspberries', 'granulated sugar']",2,5690,49598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132360,2018-12-05,5,Delicious quick thick chocolate sauce with ing...,easy microwave hot fudge topping,10,106436,2004-01-28,"['15-minutes-or-less', 'time-to-make', 'course...","[177.1, 9.0, 100.0, 3.0, 1.0, 18.0, 10.0]",7,"['mix dry ingredients', 'stir in room temperat...",this is an easy and delicious way to get a cho...,"['sugar', 'cocoa', 'cornstarch', 'salt', 'wate...",7,215681,33795
1132361,2007-04-26,0,I am not going to rate this because I did have...,garlic clove chicken,65,49304,2003-02-21,"['weeknight', 'time-to-make', 'course', 'main-...","[338.3, 32.0, 1.0, 7.0, 53.0, 29.0, 2.0]",10,"['preheat oven to 325 degrees', 'spray a large...","garlic and chicken what could be better, the c...","['chicken', 'garlic', 'dry white wine', 'lemon...",6,11621,22415
1132364,2008-06-23,5,WOW! Sometimes I don't take the time to rate ...,pot roast with port stove top,115,108291,2003-12-05,"['weeknight', 'time-to-make', 'course', 'prepa...","[828.2, 87.0, 22.0, 20.0, 93.0, 112.0, 4.0]",8,"['in a large pan , heat oil and brown roast on...",this is a recipe from the frugal gourmet cooki...,"['boneless beef chuck roast', 'olive oil', 'ta...",10,17831,32160
1132365,2009-01-11,4,Very good! I used regular port as well. The ...,pot roast with port stove top,115,108291,2003-12-05,"['weeknight', 'time-to-make', 'course', 'prepa...","[828.2, 87.0, 22.0, 20.0, 93.0, 112.0, 4.0]",8,"['in a large pan , heat oil and brown roast on...",this is a recipe from the frugal gourmet cooki...,"['boneless beef chuck roast', 'olive oil', 'ta...",10,5947,32160


Trancates dataset size for proper runs.

In [8]:
data = data.drop(data.index[150000:])

Calculate transposed matrix

In [16]:
tmat = data.pivot_table(index = 'user_id', columns = 'recipe_id',values = 'rating').fillna(0)

Apply KNN to train our model

In [17]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(tmat.values)
user_neigh_dist, user_neigh_ind = knn.kneighbors(tmat.values, n_neighbors=6)
user_neigh_ind, user_neigh_dist

(array([[    0, 36197, 13474, 35677, 31509, 13303],
        [    1, 23541, 16523, 28017, 17658,  8647],
        [    2,  7859,  1719, 13140, 28748, 11980],
        ...,
        [51789,  8714, 34531, 34528, 34530, 34529],
        [34527, 34526, 34530, 34529, 34528, 34532],
        [29425, 51791, 44030, 31429, 45898, 26819]]),
 array([[0.00000000e+00, 7.54854831e-01, 7.54854831e-01, 7.54854831e-01,
         7.54854831e-01, 7.54854831e-01],
        [7.77156117e-16, 8.46297276e-01, 8.55087682e-01, 8.55087682e-01,
         8.56515424e-01, 8.69578913e-01],
        [0.00000000e+00, 4.54455274e-01, 4.54455274e-01, 5.63564220e-01,
         5.63564220e-01, 5.63564220e-01],
        ...,
        [0.00000000e+00, 9.00031235e-01, 1.00000000e+00, 1.00000000e+00,
         1.00000000e+00, 1.00000000e+00],
        [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
         1.00000000e+00, 1.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.0

In [18]:
def findSimilarUsers(user, n=5):
        # get neigh_users_dist, neigh_users_ind 
        neigh_users_dist, neigh_users_ind = knn.kneighbors(
            np.asarray([tmat.values[user - 1]]), n_neighbors=n + 1)
        # header helper print
        print('The top ' + str(n) +  ' most similar users of user ' + str(user) + ' are:')
        # iterate all and print the distances
        for i in range(1, len(neigh_users_dist[0])):
            print('No.' + str(i) + ": User ID: " + str(neigh_users_ind[0][i]+1) + ", with distance " + str(neigh_users_dist[0][i]))

        print("\n")

        return neigh_users_dist.flatten()[1:], neigh_users_ind.flatten()[1:] + 1

In [19]:
def getRecommendations(num_recipes_recommended, avg_rating, userId):
        # clean up the zero ratings
        zero_rating = np.where(avg_rating == 0)[0][-1]
        ranked_ind = np.argsort(avg_rating)[::-1]
        ranked_ind = ranked_ind[:list(ranked_ind).index(zero_rating)]
        # check input validity by comparing with recipes we have
        num_recipes_recommended = min(len(ranked_ind), num_recipes_recommended)
        # store seen recipes in a list
        seen = list(data[data['user_id'] == userId]['name'])
        recipes = list(tmat.columns[ranked_ind])
        # recommended count
        count = 0
        # store recommended recipes in the list
        recommended_recipes = []
        for recipe in recipes:
            # only append if not seen
            if recipe not in seen:
                recommended_recipes.append(recipe)
                count += 1
            if count == num_recipes_recommended:
                break

        pprint(recommended_recipes)

In [20]:
def recommend(userId, num_similar_users, num_recipes_recommended):

    print("User " + str(userId) + " has rated the following recipes: ")
    pprint(list(data[data['user_id'] == userId]['name']))
    print("\n")
    
    # retrieve neigh_users_dist and neigh_users_ind
    neigh_users_dist, neigh_users_ind = findSimilarUsers(userId, num_similar_users)
    # weight each distance based on the total distances
    weighted_user_neigh_dist = neigh_users_dist / np.sum(neigh_users_dist)
    # Broadcasting
    weighted_user_neigh_dist = weighted_user_neigh_dist[:, np.newaxis] + np.zeros(len(tmat.columns))
    # Calculate the average rating
    avg_rating =  (weighted_user_neigh_dist * tmat.values[neigh_users_ind]).sum(axis=0)
    # helper print function
    print("Based on other users rating, we recommend:")
    
    getRecommendations(num_recipes_recommended, avg_rating, userId)


Unit Test

In [21]:
recommend(userId = 3787, num_similar_users = 5, num_recipes_recommended = 10)

User 3787 has rated the following recipes: 
['white bean   green chile pepper soup',
 'black beans with mango sauce',
 'warm spinach salad',
 'roasted asparagus   shiitake mushrooms',
 'low carb key lime pie',
 'slow cooker hoppin  john',
 'west indian pumpkin soup']


The top 5 most similar users of user 3787 are:
No.1: User ID: 36407, with distance 0.7137008328430658
No.2: User ID: 47515, with distance 0.7137008328430658
No.3: User ID: 31830, with distance 0.7137008328430658
No.4: User ID: 49158, with distance 0.7137008328430658
No.5: User ID: 25065, with distance 0.7137008328430658


Based on other users rating, we recommend:
[130403, 52424, 83971, 10291, 36581, 43817, 43768, 43724, 43728, 43746]
